# Deploy Logic Apps

To operationalize our batch scoring workflow, we need a way to trigger our pipeline. Since we're applying style transfer to video data, lets trigger the pipeline everytime a new video is uploaded and detected. To do this, we'll need a mechanism that can detect the appearance of new video data. 

Logic Apps can solve this problem for us. In this notebook, we'll deploy a pre-built logic app that will look for new videos that appear in a specified storage location. When a new video is detected, the logic app will send an http request to the published pipeline (which we deployed in the previous notebook). 

---

Import the packages we need.

In [1]:
from azureml.core import Workspace, Run, Experiment
from azureml.pipeline.core import PublishedPipeline
from azureml.core.datastore import Datastore
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from azureml.core.authentication import AzureCliAuthentication
from pathlib import Path
import re
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

Load our workspace from the config file.

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# Also create a Project and attach to Workspace
project_folder = "scripts"
run_history_name = project_folder

if not os.path.isdir(project_folder):
    os.mkdir(project_folder)

Get authentication information about our published pipeline so that we can use it during the deployment.

In [4]:
published_pipeline = PublishedPipeline.get(ws, id=get_key(env_path, "AML_PUBLISHED_PIPELINE_ID"))

In [5]:
cli_auth = AzureCliAuthentication()
aad_token = cli_auth.get_authentication_header()

### Deploy Logic App

![Logic Apps](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_logic_app.jpg)

The *logic* behind the Logic App deployment is shown above:
1. When a blob is added, begin the workflow.
2. Check the blob name. 
    - if the blob name ends with `.mp4`:
        - make a request to the AKS endpoint
    - otherwise:
        - terminate in cancellation


Create the deployment for the Azure blob storage connector.



In [6]:
!az group deployment create \
    --name blob_connector \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --template-file template.blob_connector.json \
    --parameters \
        location={get_key(env_path, "REGION")} \
        subscription_id={get_key(env_path, "SUBSCRIPTION_ID")} \
        storage_account_name={get_key(env_path, "STORAGE_ACCOUNT_NAME")} \
        storage_account_key={get_key(env_path, "STORAGE_ACCOUNT_KEY")}

{- Finished ..
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/jiataamltest02/providers/Microsoft.Resources/deployments/blob_connector",
  "location": null,
  "name": "blob_connector",
  "properties": {
    "correlationId": "1a6e909e-adb2-4390-9498-e2a37d6274ac",
    "debugSetting": null,
    "dependencies": [],
    "duration": "PT4.7165887S",
    "mode": "Incremental",
    "onErrorDeployment": null,
    "outputResources": [
      {
        "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/jiataamltest02/providers/Microsoft.Web/connections/azureblob",
        "resourceGroup": "jiataamltest02"
      }
    ],
    "outputs": null,
    "parameters": {
      "location": {
        "type": "String",
        "value": "eastus"
      },
      "storage_account_key": {
        "type": "String",
        "value": "uOpI4HeCmvKRzHSbo8qQcQe9z/LvxnIdA+f8oh5R3LlqTW6g3EhOWsn7BKgXq3otHWgdHtrf52QrBrVv+es62A=="
      },
      "storage_account_name": {
     

Create the deployment for the Logic App.

In [9]:
!az group deployment create \
    --name logic_app \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --template-file template.logic_app.json \
    --parameters \
        name="logic_app" \
        location={get_key(env_path, "REGION")} \
        resource_group={get_key(env_path, "RESOURCE_GROUP")} \
        subscription_id={get_key(env_path, "SUBSCRIPTION_ID")} \
        storage_container_name={get_key(env_path, "STORAGE_CONTAINER_NAME")} \
        url_endpoint={published_pipeline.endpoint} \
        aad_token='{aad_token["Authorization"]}' \
        datastore_name={get_key(env_path, "AML_DATASTORE_NAME")} \
        experiment_name="logic_app_experiment"

{- Finished ..
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/jiataamltest02/providers/Microsoft.Resources/deployments/logic_app",
  "location": null,
  "name": "logic_app",
  "properties": {
    "correlationId": "ab0c3b91-1235-4afe-a68f-0fa70f8f0a31",
    "debugSetting": null,
    "dependencies": [],
    "duration": "PT6.961205S",
    "mode": "Incremental",
    "onErrorDeployment": null,
    "outputResources": [
      {
        "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/jiataamltest02/providers/Microsoft.Logic/workflows/logic_app",
        "resourceGroup": "jiataamltest02"
      }
    ],
    "outputs": null,
    "parameters": {
      "aad_token": {
        "type": "String",
        "value": "Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Im5iQ3dXMTF3M1hrQi14VWFYd0tSU0xqTUhHUSIsImtpZCI6Im5iQ3dXMTF3M1hrQi14VWFYd0tSU0xqTUhHUSJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuY29yZS53aW5kb3dzLm5ldC8iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm

### Trigger logic app by adding a new video to the Azure blob container

Before testing the logic app by added a new video the blob container, check in the portal or cli that the logic app deployment has completed and that it looks correct.

In [10]:
!cp orangutan.mp4 trigger_test_orangutan.mp4

In [11]:
my_datastore = Datastore.register_azure_blob_container(
    workspace=ws, 
    datastore_name=get_key(env_path, "AML_DATASTORE_NAME"), 
    container_name=get_key(env_path, "STORAGE_CONTAINER_NAME"), 
    account_name=get_key(env_path, "STORAGE_ACCOUNT_NAME"), 
    account_key=get_key(env_path, "STORAGE_ACCOUNT_KEY"),
    overwrite=True
)

In [12]:
# Upload new trigger file video
my_datastore.upload_files(
    ["./trigger_test_orangutan.mp4"],
    overwrite=True
)

$AZUREML_DATAREFERENCE_datastore

The appearance of the new `trigger_test_orangutan.mp4` video will trigger the Logic App flow. Inspect your logic app in the portal to see the progress.

---

You are now ready to move on to the [next notebook](05_clean_up.ipynb).